In [108]:
import datetime
import os
import sys
import pandas as pd
import FinanceDataReader as fdr
import OpenDartReader
import numpy as np

from tqdm import tqdm
from tabulate import tabulate
from datetime import timedelta

df_kospi = fdr.StockListing('KOSPI')
df_kosdaq = fdr.StockListing('KOSDAQ')

# 관리종목
df_AdmStock = fdr.StockListing('KRX-ADMIN')

df_krx = pd.concat([df_kospi, df_kosdaq])

# 관리종목 및 우선주 제거(5, 7, 9, K)
df_filterStk = df_krx[~df_krx['Code'].isin(df_AdmStock['Symbol'])]
df_filterStk = df_filterStk[~df_filterStk['Code'].str.endswith(('5', '7', '9', 'K'))]

df_filterStk[0:10]

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,81500,2,-100,-0.12,81900,81900,80800,9455929,768935755446,486537277825000,5969782550,STK
1,000660,KR7000660001,SK하이닉스,KOSPI,,236500,3,0,0.00,235500,237500,232500,2368287,557342030500,172172559322500,728002365,STK
2,373220,KR7373220003,LG에너지솔루션,KOSPI,,326500,2,-1500,-0.46,327000,331500,322500,152685,49790949000,76401000000000,234000000,STK
3,005380,KR7005380001,현대차,KOSPI,,295000,2,-3000,-1.01,299500,299500,293000,761453,225020078500,61777776345000,209416191,STK
5,207940,KR7207940008,삼성바이오로직스,KOSPI,,727000,2,-5000,-0.68,732000,734000,724000,76001,55208455000,51743498000000,71174000,STK
6,000270,KR7000270009,기아,KOSPI,,129300,2,-2200,-1.67,132300,132300,128600,1026360,133000579400,51701693318100,399858417,STK
7,068270,KR7068270008,셀트리온,KOSPI,,174300,1,500,0.29,174100,175400,173500,331353,57766742200,37821918768900,216993223,STK
8,105560,KR7105560007,KB금융,KOSPI,,78500,3,0,0.00,79300,79900,78200,1017673,80134672600,31675619152000,403511072,STK
9,005490,KR7005490008,POSCO홀딩스,KOSPI,,363000,1,3500,0.97,358500,365000,355000,271140,97750385500,30699356490000,84571230,STK
10,035420,KR7035420009,NAVER,KOSPI,,166900,1,1500,0.91,167300,169000,165900,709968,118607248300,27105994338600,162408594,STK


In [109]:
initDate = datetime.date(2018, 12, 28)
bfDate = initDate + timedelta(days=-180)    # 6개월 전 날짜(2018-07-01)

# Create lists to store results
bf_half_year_dates = []
bf_half_year_closes = []
half_year_ratios = []

# 종목 별 6개월전 주가 / 6개월전 시점 / 6개월 수익률 계산
for index, row in tqdm(df_filterStk.iterrows(), total=df_filterStk.shape[0]):
    stkCode = row['Code']
    
    try:
        # 주가 데이터 조회
        df_stockPrice = fdr.DataReader(stkCode, bfDate, initDate)

        # 해당 날짜에 데이터가 있는 경우에만 진행11
        if df_stockPrice.shape[0] > 0:
            bfPrice = df_stockPrice.iloc[0]['Close']
            currPrice = df_stockPrice.iloc[-1]['Close']
            
            # Calculate values
            bf_half_year_dates.append(bfDate)
            bf_half_year_closes.append(bfPrice)
            half_year_ratios.append((currPrice - bfPrice) / bfPrice * 100)
        else:
            # 해당 시점에 주가 없으면 None 처리
            bf_half_year_dates.append(None)
            bf_half_year_closes.append(None)
            half_year_ratios.append(None)
    except Exception as e:
        print(f'{e.args}')
        bf_half_year_dates.append(None)
        bf_half_year_closes.append(None)
        half_year_ratios.append(None)

df_tmp = pd.DataFrame({
    'Code': df_filterStk['Code'],
    'BF_HALF_YEAR_DATE': bf_half_year_dates,
    'BF_HALF_YEAR_CLOSE': bf_half_year_closes,
    'HALF_YEAR_RATIO': half_year_ratios
})

# 디스플레이 욥션
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

100%|██████████| 2488/2488 [02:57<00:00, 14.05it/s]


In [110]:
# 기존 종목 리스트에 6개월 데이터 추가
df_tmp1 = pd.concat([df_filterStk, df_tmp.drop(columns=['Code'])], axis=1)


In [111]:
# 수익률 기준으로 오름차순 정렬
df_tmp1 = df_tmp1.dropna(subset=['BF_HALF_YEAR_DATE'])

df_tmp2 = df_tmp1.sort_values(by='HALF_YEAR_RATIO', ascending=False)
df_tmp2

df_result = df_tmp2[0:50]
df_result

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId,BF_HALF_YEAR_DATE,BF_HALF_YEAR_CLOSE,HALF_YEAR_RATIO
801,051630,KR7051630002,진양화학,KOSPI,,3530,1,90,2.62,3490,3655,3445,7499,26403925,55774000000,15800000,STK,2018-07-01,1939.0,199.948427
828,009460,KR7009460007,한창제지,KOSPI,,786,2,-1,-0.13,787,793,778,52582,41203114,46898643996,59667486,STK,2018-07-01,996.0,198.192771
852,238090,KR7238090005,앤디포스,KOSDAQ,벤처기업부,4055,1,20,0.50,3965,4120,3960,11298,45940660,95459983665,23541303,KSQ,2018-07-01,4939.0,194.270095
500,143160,KR7143160000,아이디스,KOSDAQ,우량기업부,15940,2,-30,-0.19,16010,16010,15710,20352,322764420,170819256600,10716390,KSQ,2018-07-01,7210.0,193.342580
403,002320,KR7002320000,한진,KOSPI,,19580,1,170,0.88,19410,19650,19390,11306,221116710,292674556240,14947628,STK,2018-07-01,19093.0,168.333944
486,008350,KR7008350001,남선알미늄,KOSPI,,1706,2,-4,-0.23,1694,1709,1694,308878,525297935,220208927540,129079090,STK,2018-07-01,1110.0,154.954955
146,025980,KR7025980004,아난티,KOSDAQ,우량기업부,5580,1,80,1.45,5500,5620,5490,211977,1176795310,494552487240,88629478,KSQ,2018-07-01,7600.0,146.710526
1054,950110,KR8392070007,SBI핀테크솔루션즈,KOSDAQ,외국기업(소속부없음),3005,3,0,0.00,3005,3040,2950,6907,20738875,72277882700,24052540,KSQ,2018-07-01,6870.0,137.263464
1257,263920,KR7263920001,휴엠앤씨,KOSDAQ,벤처기업부,1092,2,-5,-0.46,1108,1108,1085,53129,57859518,53557286328,49045134,KSQ,2018-07-01,4179.0,134.290500
758,000890,KR7000890004,보해양조,KOSPI,,485,1,5,1.04,481,491,480,272137,131999304,67473262565,139120129,STK,2018-07-01,850.0,122.352941


In [112]:
fs = dart.finstate('005930', 2024, 11013)
net_income_amounts = fs[(fs['fs_div'] == 'CFS') & (fs['sj_div'] == 'IS') & (fs['account_nm'] == '당기순이익')]['thstrm_amount']
net_income_amounts.iloc[0]

'6,754,708,000,000'

In [136]:

api_key = 'd00cc7609f43e2b2e6b560e039442fe3751dbde7'
dart = OpenDartReader(api_key)

# 1분기: 11013, 2분기: 11012, 3분기: 11014, 4분기: 11011
# CFS:연결재무제표 OFS:재무제표
# BS:재무상태표, IS:손익계산서
# fs = dart.finstate('005930', 2019, 11011)

quarter_info = {
        'Q1': {
            'prev_quarters': [11011, 11014],
            'tempYear': [initDate.year - 1, initDate.year - 1]  # 4분기, 3분기 모두 전년도
        },
        'Q2': {
            'prev_quarters': [11013, 11011],
            'tempYear': [initDate.year, initDate.year - 1]  # 1분기 현재년도, 4분기 전년도
        },
        'Q3': {
            'prev_quarters': [11012, 11013],
            'tempYear': [initDate.year, initDate.year]  # 2분기, 1분기 모두 현재년도
        },
        'Q4': {
            'prev_quarters': [11014, 11012],
            'tempYear': [initDate.year, initDate.year]  # 3분기, 2분기 모두 현재년도
        }
    }

# 현재 날짜의 직진 4개분기 구하기
if initDate.month in [1, 2, 3]:
    qty = 'Q1'
elif initDate.month in [4, 5, 6]:
    qty = 'Q2'
elif initDate.month in [7, 8, 9]:
    qty = 'Q3'
else: # 10, 11, 12
    qty = 'Q4'

prev_quarters_info = quarter_info[qty]

# 결과를 저장할 리스트
results = []

for col, ser in df_result.iterrows():
    for i, code in enumerate(prev_quarters_info['prev_quarters']):
        year = prev_quarters_info['tempYear'][i]
        try:
            fs = dart.finstate(ser['Code'], year, code)
            if len(fs.columns) > 0:
                net_incomes = fs[(fs['fs_div'] == 'OFS') & (fs['sj_div'] == 'IS') & (fs['account_nm'] == '당기순이익')]['thstrm_amount']
                if not net_incomes.empty:
                    # val = net_incomes.iloc[0]
                    all_positive = all(pd.to_numeric(net_incomes, errors='coerce') >= 0)
                    if not all_positive:
                        include_in_results = False
                    else:
                        val = net_incomes.iloc[0]
                else:
                    val = np.nan
        except KeyError as e:
            print(f"KeyError occurred: {e}")
            
    if include_in_results:
        result = {
            'Code': ser['Code'],
            'NetIncome': val if 'val' in locals() else np.nan
        }
        results.append(result)


# DataFrame 생성
df_fund = pd.DataFrame(results)

# NaN 값 제거
df_fund = df_fund.dropna(subset=['NetIncome'])

# 중복 제거
df_fund = df_fund.drop_duplicates()

df_fund


{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}



""


In [ ]:
df_result = pd.concat([df_result, df_fund.drop(columns=['Code'])], axis=1)
df_result

,Code,ISU_CD,Name_x,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId,BF_HALF_YEAR_DATE,BF_HALF_YEAR_CLOSE,HALF_YEAR_RATIO,Name_y,Year,Quarter,NetIncome,Quarter,NetIncome
0,051630,KR7051630002,진양화학,KOSPI,,3530,1,90.0,2.62,3490.0,3655.0,3445.0,7499.0,2.640392e+07,5.577400e+10,15800000.0,STK,2018-07-01,1939.0,199.948427,진양화학,2018.0,11014.0,"-232,537,414",11014,"-232,537,414"
1,051630,KR7051630002,진양화학,KOSPI,,3530,1,90.0,2.62,3490.0,3655.0,3445.0,7499.0,2.640392e+07,5.577400e+10,15800000.0,STK,2018-07-01,1939.0,199.948427,진양화학,2018.0,11012.0,"109,974,738",11012,"109,974,738"
2,009460,KR7009460007,한창제지,KOSPI,,786,2,-1.0,-0.13,787.0,793.0,778.0,52582.0,4.120311e+07,4.689864e+10,59667486.0,STK,2018-07-01,996.0,198.192771,한창제지,2018.0,11014.0,"1,699,496,023",11014,"-232,537,414"
3,009460,KR7009460007,한창제지,KOSPI,,786,2,-1.0,-0.13,787.0,793.0,778.0,52582.0,4.120311e+07,4.689864e+10,59667486.0,STK,2018-07-01,996.0,198.192771,한창제지,2018.0,11012.0,"2,476,267,941",11012,"109,974,738"
4,238090,KR7238090005,앤디포스,KOSDAQ,벤처기업부,4055,1,20.0,0.50,3965.0,4120.0,3960.0,11298.0,4.594066e+07,9.545998e+10,23541303.0,KSQ,2018-07-01,4939.0,194.270095,앤디포스,2018.0,11014.0,"2,540,899,732",11014,"1,699,496,023"
5,238090,KR7238090005,앤디포스,KOSDAQ,벤처기업부,4055,1,20.0,0.50,3965.0,4120.0,3960.0,11298.0,4.594066e+07,9.545998e+10,23541303.0,KSQ,2018-07-01,4939.0,194.270095,앤디포스,2018.0,11012.0,"396,125,984",11012,"2,476,267,941"
6,143160,KR7143160000,아이디스,KOSDAQ,우량기업부,15940,2,-30.0,-0.19,16010.0,16010.0,15710.0,20352.0,3.227644e+08,1.708193e+11,10716390.0,KSQ,2018-07-01,7210.0,193.342580,아이디스,2018.0,11014.0,"1,392,401,458",11014,"1,699,496,023"
7,143160,KR7143160000,아이디스,KOSDAQ,우량기업부,15940,2,-30.0,-0.19,16010.0,16010.0,15710.0,20352.0,3.227644e+08,1.708193e+11,10716390.0,KSQ,2018-07-01,7210.0,193.342580,아이디스,2018.0,11012.0,"963,450,274",11012,"2,476,267,941"
8,002320,KR7002320000,한진,KOSPI,,19580,1,170.0,0.88,19410.0,19650.0,19390.0,11306.0,2.211167e+08,2.926746e+11,14947628.0,STK,2018-07-01,19093.0,168.333944,한진,2018.0,11014.0,"-6,564,572,029",11014,"2,540,899,732"
9,002320,KR7002320000,한진,KOSPI,,19580,1,170.0,0.88,19410.0,19650.0,19390.0,11306.0,2.211167e+08,2.926746e+11,14947628.0,STK,2018-07-01,19093.0,168.333944,한진,2018.0,11012.0,"1,427,464,891",11012,"396,125,984"


In [137]:
# 1분기: 11013, 2분기: 11012, 3분기: 11014, 4분기: 11011
# CFS:연결재무제표 OFS:재무제표
# BS:재무상태표, IS:손익계산서
# fs = dart.finstate('005930', 2019, 11011)

api_key = 'd00cc7609f43e2b2e6b560e039442fe3751dbde7'
dart = OpenDartReader(api_key)

# fs = dart.finstate('005930', 2024)
fs = dart.finstate('005930', 2024, 11013)
fs

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,ord,currency,thstrm_add_amount,frmtrm_add_amount
0,20240516001421,11013,2024,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동자산,제 56 기1분기말,2024.03.31 현재,"208,544,280,000,000",제 55 기말,2023.12.31 현재,"195,936,557,000,000",1,KRW,NaN,NaN
1,20240516001421,11013,2024,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동자산,제 56 기1분기말,2024.03.31 현재,"262,355,532,000,000",제 55 기말,2023.12.31 현재,"259,969,423,000,000",3,KRW,NaN,NaN
2,20240516001421,11013,2024,00126380,005930,CFS,연결재무제표,BS,재무상태표,자산총계,제 56 기1분기말,2024.03.31 현재,"470,899,812,000,000",제 55 기말,2023.12.31 현재,"455,905,980,000,000",5,KRW,NaN,NaN
3,20240516001421,11013,2024,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동부채,제 56 기1분기말,2024.03.31 현재,"81,770,355,000,000",제 55 기말,2023.12.31 현재,"75,719,452,000,000",7,KRW,NaN,NaN
4,20240516001421,11013,2024,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동부채,제 56 기1분기말,2024.03.31 현재,"17,213,333,000,000",제 55 기말,2023.12.31 현재,"16,508,663,000,000",9,KRW,NaN,NaN
5,20240516001421,11013,2024,00126380,005930,CFS,연결재무제표,BS,재무상태표,부채총계,제 56 기1분기말,2024.03.31 현재,"98,983,688,000,000",제 55 기말,2023.12.31 현재,"92,228,115,000,000",11,KRW,NaN,NaN
6,20240516001421,11013,2024,00126380,005930,CFS,연결재무제표,BS,재무상태표,자본금,제 56 기1분기말,2024.03.31 현재,"897,514,000,000",제 55 기말,2023.12.31 현재,"897,514,000,000",13,KRW,NaN,NaN
7,20240516001421,11013,2024,00126380,005930,CFS,연결재무제표,BS,재무상태표,이익잉여금,제 56 기1분기말,2024.03.31 현재,"350,847,854,000,000",제 55 기말,2023.12.31 현재,"346,652,238,000,000",17,KRW,NaN,NaN
8,20240516001421,11013,2024,00126380,005930,CFS,연결재무제표,BS,재무상태표,자본총계,제 56 기1분기말,2024.03.31 현재,"371,916,124,000,000",제 55 기말,2023.12.31 현재,"363,677,865,000,000",21,KRW,NaN,NaN
9,20240516001421,11013,2024,00126380,005930,CFS,연결재무제표,IS,손익계산서,매출액,제 56 기1분기,2024.01.01 ~ 2024.03.31,"71,915,601,000,000",제 55 기1분기,2023.01.01 ~ 2023.03.31,"63,745,371,000,000",23,KRW,"71,915,601,000,000","63,745,371,000,000"
